<a href="https://colab.research.google.com/github/fhdbbk/learning_tf2/blob/master/advanced_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 2 quickstart for experts

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/advanced"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/advanced.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This is a [Google Colaboratory](https://colab.research.google.com/notebooks/welcome.ipynb) notebook file. Python programs are run directly in the browser—a great way to learn and use TensorFlow. To follow this tutorial, run the notebook in Google Colab by clicking the button at the top of this page.

1. In Colab, connect to a Python runtime: At the top-right of the menu bar, select *CONNECT*.
2. Run all the notebook code cells: Select *Runtime* > *Run all*.

Download and install TensorFlow 2. Import TensorFlow into your program:

Note: Upgrade `pip` to install the TensorFlow 2 package. See the [install guide](https://www.tensorflow.org/install) for details.

Import TensorFlow into your program:

In [0]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
assert(len(x_train) == len(y_train))
assert(len(x_test) == len(y_test))
print(f"Number of training images: {len(x_train)}")
print(f"Number of test images: {len(x_test)}")
print(f"Shape of images: {x_train.shape}")
print(f"Shape of labels: {y_train.shape}")

Number of training images: 60000
Number of test images: 10000
Shape of images: (60000, 28, 28, 1)
Shape of labels: (60000,)


Understanding `tf.data.Dataset`

In [14]:
ds = tf.data.Dataset.range(10)
print(list(ds.as_numpy_iterator()))
ds1 = ds.shuffle(5)                   # Creates a buffer of size 5 from this ds, fills it with 5 elements after shuffling
print(list(ds1.as_numpy_iterator()))
ds2 = ds1.batch(3)                    # Randomly samples 3 elements from ds1's buffer and those buffer elements are replaced by remaining elements from the ds
print(list(ds2.as_numpy_iterator()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 3, 0, 4, 7, 5, 1, 9, 6, 8]
[array([2, 4, 1]), array([3, 7, 9]), array([0, 5, 6]), array([8])]


Use `tf.data` to batch and shuffle the dataset:

In [0]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

Build the `tf.keras` model using the Keras [model subclassing API](https://www.tensorflow.org/guide/keras#model_subclassing):

In [0]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Choose an optimizer and loss function for training: 

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()      # Adam uses a default lr of 0.001

Understanding `tf.keras.metrics.Mean` and `tf.keras.metrics.SparseCategoricalAccuracy`

In [29]:
m = tf.keras.metrics.Mean()
r = m.update_state([1, 3, 5, 7])
print(r.value().numpy())
print(m.result())

m.reset_states()
m.update_state([1, 3, 5, 7], sample_weight=[1, 1, 0, 0])
print(f"Weighted average: {m.result().numpy()}")

m = tf.keras.metrics.SparseCategoricalAccuracy()
                # y_true              # y_pred logits
m.update_state([[2], [1]], [[0.1, 0.9, 0.8], [0.05, 0.95, 0]])
m.result().numpy()

4.0
tf.Tensor(4.0, shape=(), dtype=float32)
Weighted average: 2.0


0.5

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [0]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)      # calculates dL(w1, w2, ..., wn)/dwi   for i = 1 to n
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))  #calculates wi = wi - lr*dwi

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [0]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [39]:
tf.keras.backend.clear_session()
EPOCHS = 5
for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {:5f}, Accuracy: {:5f}, Test Loss: {:5f}, Test Accuracy: {:5f}'
  print(template.format(epoch + 1,
                        train_loss.result().numpy(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))

Epoch 1, Loss: 0.006628, Accuracy: 99.783333, Test Loss: 0.057277, Test Accuracy: 98.570000
Epoch 2, Loss: 0.005027, Accuracy: 99.836662, Test Loss: 0.071594, Test Accuracy: 98.500000
Epoch 3, Loss: 0.005031, Accuracy: 99.826660, Test Loss: 0.076035, Test Accuracy: 98.589996
Epoch 4, Loss: 0.003584, Accuracy: 99.881668, Test Loss: 0.068696, Test Accuracy: 98.589996
Epoch 5, Loss: 0.003006, Accuracy: 99.904999, Test Loss: 0.074832, Test Accuracy: 98.409996


In [0]:
import numpy as np

In [38]:
res1 = model(np.expand_dims(x_test[0], axis=0)).numpy()
print(res1)
lbl = np.argmax(res1, axis=1)
print(lbl)
print(lbl.squeeze())
print("Actual label: {}".format(y_test[0]))

[[ -7.064634   -4.045855   -3.2238052  -0.5918914  -8.267031   -4.3813148
  -13.820008   22.859444   -5.0845184   2.7319942]]
[7]
7
Actual label: 7


The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials).